In [1]:
import sys
sys.path.append('/home/gregor/Documents/UNI/WS25/NSSC1/Schöberl/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [2]:
#To test for a chain of elements
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

use_constraint = True


number_of_masses = 10
starting_position = (0,0,0)
length_chain = 10
single_length = length_chain/number_of_masses

prev_obj = mss.add(Fix(starting_position))
for i in range(number_of_masses):
    pos = (starting_position[0] + (i + 1) * single_length, starting_position[1], starting_position[2])
    new_mass = mss.add(Mass(1.0, pos)) 
    mss.add(Spring(1, 10000, (prev_obj, new_mass)))
    prev_obj = new_mass# 

In [3]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [4]:
view_width = 800
view_height = 400

camera = PerspectiveCamera( position=[0, -20, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=2.0, position=(0.0, -20.0, 0.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [6]:
from time import sleep
for i in range(1000):
    mss.simulate(0.01, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 